<style>
@font-face {font-family: "B Nazanin"; src: url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.eot"); src: url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.eot?#iefix") format("embedded-opentype"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.woff2") format("woff2"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.woff") format("woff"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.ttf") format("truetype"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.svg#B Nazanin") format("svg"); }
    </style>
<div dir="rtl" align="center" style ='font-family: "B Nazanin";'>
    <h1>
        تمرین سوم درس پردازش زبان‌های طبیعی
    </h1>
    <h3>
        گردآورندگان:<br/>
        ساحل مس‌فروش، سروش تابش، درنا دهقانی
    </h3>
    
</div>

<br/>
<br/>


<div dir="rtl"  style ='font-family: "B Nazanin";'>
    ما در این تمرین ترک "تکمیل مصراع دوم با رعایت وزن شعر" را انتخاب کردیم. در این تمرین قصد داریم به کمک مدل‌های زبانی مناسب، چند کلمه به عنوان یک مصرع از بیت را به عنوان ورودی بگیریم و چند کلمه به عنوان مصرع دوم این بیت را خروجی دهیم. در این تمرین ما از وبسایت گنجور برای جمع‌آوری داده کمک گرفتیم و از چندین قالب شعری استفاده کردیم، زیرا قافیه داشتن یا نداشتن به عنوان ورودی داده می‌شود. اگر نیاز به هم‌قافیه بودن دو مصراع بود، از قالب مثنوی نظیر شاهنامه‌ی فردوسی کمک می‌گیریم و اگر نیاز به هم‌قافیه بودن دو مصراع نبود می‌توانیم از قالب غزل نظیر اشعار حافظ استفاده کنیم. <br>
    برای ساخت چنین سیستمی از چند مدل زبانی استفاده می‌کنیم: مدل n-gram، مدل encoder-decoder با یک شبکه LSTM به عنوان encoder، و مدلی پیرو مکانیسم توجه.
</div>

In [2]:
! pip install pandas
! pip install hazm
! pip install transformers
! pip install torch
! pip install nltk
! pip install numpy
! pip install codec
! pip install wandb
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 13.2 MB/s 
     |████████████████████████████████| 233 kB 66.9 MB/s 
     |████████████████████████████████| 1.4 MB 65.4 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=83f9fb7c7c82fd27f58b62f76e39986a2c49d02bf4f7016db347014f05caac7f
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=153858 sha256=84f28949d5f95f2385c48f838258f35cd0ab77a29dba92319cb7f11b39e7a51b
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk

In [4]:
import random 
import numpy as np
import nltk
import pandas as pd
import codecs
from tqdm import tqdm
import os
import re
import glob
from __future__ import unicode_literals
from hazm import *
from nltk import FreqDist
import itertools
import pandas as pd
import torch
import wandb
import time
import copy
from collections import Counter

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    <h3> 1. دوباره‌نویسی داده </h3>
    از آنجایی که در داده‌ی گرفته شده، ابتدا و انتهای ابیات و مصراع‌ها مشخص نبود، به کمک کد زیر این داده را بازنویسی کردیم. ابتدای هر مصرع جدید با __BOM__ و پایان بیت با __EOM__ مشخص گردید.</div>

In [ ]:
path = 'all_norm.txt'
beyt_file = ''
with open(path, 'r', encoding="utf-8") as fp:
    lines = fp.readlines()
    for i in range(0, len(lines) - 1, 2):
        mesra1 = '__BOM__ ' + lines[i].strip()
        mesra2 = '__BOM__ ' + lines[i + 1].strip()
        b = mesra1.strip() + ' ' + mesra2.strip() + ' __EOM__\n'
        beyt_file += b
        
with open('all_beyt.txt', 'w', encoding="utf-8") as fp:
    fp.write(beyt_file)

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    <h3> 2. مدل n-gram </h3>
    برای ساخت این مدل از کلاس Ngram استفاده کردیم که به کمک مطالب گفته شده در کلاس نوشته شده است. <br> در این مدل، دیتای لازم برای train، مقدار مورد استفاده برای laplace smoothing و n به عنوان ورودی گرفته می‌شوند و بر اساس آنها مدل ساخته می‌شود. هنگام train کردن، اگر با token مواجه شدیم که تکرار آن در کل دیتا بیش از 1 نبود از عبارت __UNK__ به جای آن استفاده می‌کنیم. سپس احتمال هر n-gram متناسب با تعداد آن در کل و تعداد (n_1)-gram ها محاسبه شده و laplace smoothing روی آن انجام می‌گیرد. نهایتاً n-gramها با توجه به احتمالشان به طور کاهشی مرتب می‌شوند، تا سرعت یافتن مصراع دوم بالاتر رود.<br> سپس یک مصراع به عنوان ورودی گرفته می‌شود. با توجه به n، n_1 token پایانی مصراع ورودی پیش پردازش می‌شود و بین n-gramهای موجود که شروعشان با آن n_1 token باشد، n-gram با بیشترین احتمال انتخاب شده و token پایانی آن به عنوان کلمه خروجی داده می‌شود. به همین ترتیب token تولید می‌شود تا به __EOM__ برخورد کنیم.<br>این مدل unigram را نیز اجرا می‌کند و بدین صورت عمل می‌کند که به تعداد کلمات معمول یک مصراع شاهنامه، یعنی 6، متناسب با احتمال هر unigram آنها را تولید می‌کند. واضحاً چون با این روش هر کلمه مستقل از دیگری تولید می‌شود، نتیجه قابل قبول نیست.<br>یک boolean به عنوان ورودی تابع generate_mesra برای رعایت یا عدم رعایت قافیه در نظر گرفته می‌شود. متناسب با مقدار این boolean پس از ساخت مصراع دوم، کلمه‌ی آخر آن بررسی می‌گردد؛ به طور مثال اگر این boolean مقدار True داشت و دو مصراع هم‌قافیه نبودند، کلمه‌ی آخر مصراع ساخته شده به تعداد بار محدودی مجدداً ساخته می‌شود پس از هر بار ساخت مجدداً هم‌قافیه بودن بررسی می‌گردد. در صورتی‌که پس از این تعداد بار محدود هم‌چنان دو مصراع به شرط قافیه‌ی گفته شده نرسیده بودند، دیگر به این شرط توجهی نمی‌شود و شعر ساخته شده‌ی نهایی فارغ از هم‌قافیه بودن یا نبودن خروجی داده می‌شود.<br>بررسی هم‌قافیه بودن با مقایسه‌ی 2 حرف آخر دو کلمه انجام می‌گیرد. اگر دو حرف آخر دو کلمه یکسان بودند هم‌قافیه‌اند.
</div>

In [ ]:
class Ngram(object):
    
    UNK = '__UNK__'
    BOM = '__BOM__'
    EOM = '__EOM__'
    
    def __init__(self, data, n, laplace=1):
        self.vocab = dict()
        self.n = n
        self.laplace = laplace
        self.tokens = self.preprocess(data)
        self.vocab = nltk.FreqDist(self.tokens)
        self.model = self.create_model()
    
    def preprocess(self, data):
        tokens = data.strip().split()
        if not len(self.vocab):
            self.vocab = nltk.FreqDist(tokens)
        return [token if self.vocab[token] > 1 else Ngram.UNK for token in tokens]
    
    def create_model(self):
        num_tokens = len(self.tokens)
        if self.n == 1:
            model = { (unigram,): count / num_tokens for unigram, count in self.vocab.items() }
        else:
            model = self.smooth()
        return dict(sorted(model.items(), key=lambda x: x[1], reverse=True))
    
    def smooth(self):
        vocab_len = len(self.vocab)
        n_grams = nltk.ngrams(self.tokens, self.n)
        n_vocab = nltk.FreqDist(n_grams)
        m_grams = nltk.ngrams(self.tokens, self.n - 1)
        m_vocab = nltk.FreqDist(m_grams)
        
        def counted_smoothing(n_gram, n_count):
            m_gram = n_gram[:-1]
            m_count = m_vocab[m_gram]
            return (n_count + self.laplace) / (m_count + self.laplace * vocab_len)
        
        return { n_gram: counted_smoothing(n_gram, count) for n_gram, count in n_vocab.items() }
    
    def check_ghafie(self, ghafie1, ghafie2):
        margin = 3
        ham_ghafie = False
        if len(ghafie1) > 1 and len(ghafie2) > 1:
            for i in range(-1, -margin-1, -1):
                if ghafie1[i] != ghafie2[i]:
                    break
                else:
                    ham_ghafie = True
        return ham_ghafie
        
    
    def get_candidates(self, beginning_of_ngram, last_word, ghafie, ghafie_repeat, mesra1_ghafie):
        counter = 0
        best = None
        for ng in self.model.items():
            for i in range(self.n - 1):
                if ng[0][i] != beginning_of_ngram[i]:
                    break
            else:
                if ng[0][-1] != self.BOM and ng[0][-1] != self.UNK:
                    if not last_word:
                        return ng[0][-1]
                    if counter == 0:
                        best = ng[0][-1]
                    if ((ghafie and not self.check_ghafie(mesra1_ghafie, ng[0][-1])) or (not ghafie and self.check_ghafie(mesra1_ghafie, ng[0][-1]))) and counter < ghafie_repeat:
                        counter += 1
                    else:
                        if counter < ghafie_repeat:
                            return ng[0][-1]
        if best:
            return best
        return self.UNK
    
    def generate_mesra(self, first_mesra, ghafie):
        first_mesra = self.BOM + ' ' + first_mesra + ' ' + self.BOM
        first_mesra_tokens = self.preprocess(first_mesra)
        next_mesra = []
        ghafie_repeat = 5
        mesra1_ghafie = first_mesra_tokens[-2]
                
        if self.n == 1:
            vocab_list = [x[0] for x in self.model]
            vocab_probs = list(self.model.values())
            next_mesra = list(np.random.choice(vocab_list, p=vocab_probs, size=8, replace=False))
            
            if ghafie:
                if not self.check_ghafie(first_mesra_tokens[-2], next_mesra[-1]):
                    for i in range(ghafie_repeat):
                        next_mesra[-1] = np.random.choice(vocab_list, p=vocab_probs)
                        if self.check_ghafie(first_mesra_tokens[-2], next_mesra[-1]):
                            break
            else:
                if self.check_ghafie(first_mesra_tokens[-2], next_mesra[-1]):
                    for i in range(ghafie_repeat):
                        next_mesra[-1] = np.random.choice(vocab_list, p=vocab_probs)
                        if not self.check_ghafie(first_mesra_tokens[-2], next_mesra[-1]):
                            break
                
        
        else:
            beginning_of_ngram = tuple(first_mesra_tokens[-self.n + 1:])
            last_word = False
            while True:
                next_word = self.get_candidates(beginning_of_ngram, last_word, ghafie, ghafie_repeat, mesra1_ghafie)
                
                if next_word == self.EOM:
                    last_word = True
                    next_mesra[-1] = self.get_candidates(temp_beginning_of_ngram, last_word, ghafie, ghafie_repeat, mesra1_ghafie)
                    break
        
                next_mesra.append(next_word)
                temp_beginning_of_ngram = beginning_of_ngram
                beginning_of_ngram = (*beginning_of_ngram[1:], next_word)
                
        while self.EOM in next_mesra:
            next_mesra.remove(self.EOM)
        while self.BOM in next_mesra:
            next_mesra.remove(self.BOM)
        while self.UNK in next_mesra:
            next_mesra.remove(self.UNK)
        return(' '.join(next_mesra))

In [ ]:
gram5 = Ngram(beyt_file, 5)
gram4 = Ngram(beyt_file, 4)
gram3 = Ngram(beyt_file, 3)

In [ ]:
mesra1 = 'به گرد اندر آرد بهنگام کار'
gram4.generate_mesra(mesra1, False)

'ز دینار وز گوهر نابسود'

In [ ]:
gram4.generate_mesra(mesra1, True)

'ز دینار وز گوهر شاهوار'

In [ ]:
mesra1 = 'سپاه اندر آمد به پیش سوار'
gram4.generate_mesra(mesra1, True)

'سپردار و جوشنوران صد هزار'

In [ ]:
gram4.generate_mesra(mesra1, False)

'سپردار و جوشنوران صد هزار'

In [ ]:
mesra1 = 'بران تیغ زهر آب داده به دست'
gram5.generate_mesra(mesra1, True)

'همی رفت پویان به کردار مست'

In [ ]:
mesra1 = 'هاتفی از گوشه میخانه دوش'
gram5.generate_mesra(mesra1, True)

'گفت ببخشند گنه می بنوش'

In [ ]:
mesra1 = 'هزار دشمنم ار می کنند قصد هلاک'
gram3.generate_mesra(mesra1, False)

'ز هر سو که بد مهتری با گهر'

In [ ]:
mesra1 = 'سلامی چو بوی خوش آشنایی'
gram3.generate_mesra(mesra1, False)

'بدان تا شود نزد سالار توران سپاه'

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    <h3> 3. مدل بر پایه LSTM </h3>
    ابتدا بررسی می‌کنیم که سیستم قابلیت استفاده از cuda را دارد یا نه.
</div>

In [ ]:
! pip install -q wandb
! git clone "https://github.com/amnghd/Persian_poems_corpus.git"
! mkdir "corpus"
! cp "Persian_poems_corpus/normalized/ferdousi_norm.txt" "Persian_poems_corpus/normalized/hafez_norm.txt" "Persian_poems_corpus/normalized/moulavi_norm.txt"
"./corpus/"


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# for accessing the files in google drive from google colab
from google.colab import drive
drive.mount('/content/drive')

Cloning into 'Persian_poems_corpus'...
remote: Enumerating objects: 159, done.
remote: Total 159 (delta 0), reused 0 (delta 0), pack-reused 159
Receiving objects: 100% (159/159), 45.21 MiB | 19.40 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Checking out files: 100% (148/148), done.
cp: target 'Persian_poems_corpus/normalized/moulavi_norm.txt' is not a directory
cpu
Mounted at /content/drive


<div dir="rtl"  style ='font-family: "B Nazanin";'>
    برای مانیتور کردن پارامترها و تغییرات مدل هنگام train شدن می‌توانیم از wandb استفاده کنیم. هم‌چنین امکان ذخیره‌ی پارامترها برای استفاده‌ی مجدد نیز وجود دارد. 
</div>

In [ ]:
wandb.init(project="ferdousi-generator")

wandb: Currently logged in as: thedrna (nlp400). Use `wandb login --relogin` to force relogin


In [ ]:
class Config:
    pass


wandb_active = False
project_name = 'poem_generator'
run_name = 'all_poem_train'
checkpoints_dir = '../data/checkpoints/'
corpus_dir = '../data/poems/'
vocab_path = '../data/vocabulary.txt'

if wandb_active:
    wandb.init(project=project_name, name=run_name)
    config = wandb.config
else:
    config = Config()
config.batch_size = 256
config.embedding_size = 512
config.lstm_num_layers = 3
config.lstm_hidden_size = 512
config.sequence_length = 10
config.log_interval = 10
config.learning_rate = 0.001
config.vocab_size = 38590
config.lstm_dropout = 0.2

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    در کلاس PoemDataset به خواندن و ایندکس کردن دیتا می‌پردازیم. هم‌چنین کلمات یکتای دیتا برای تشکیل vocabulary استخراج می‌گردند. برای هر کلمه‌ی یکتا، ایندکسی در نظر گرفته می‌شود و توسط تابع __getitem__ داده‌ای مناسب ورودی دادن به مدل گرفته می‌شود. از index_to_word و word_to_index در انکد و دیکد کردن استفاده می‌شود.<br>به کمک این کلاس می‌توان از میان داده‌های موجود، ابتدا به کمک تمامی اشعار  مجموعه‌ی واژگان کاملی ساخت و مدل را به کمک آن train کرد، سپس بر حسب هر شاعر آن را fine tune کرد. هم‌چنین برای پیش‌پردازش اشعار، ابتدا و انتهای مصراع‌ها و ابیات به همراه نام شاعر مشخص می‌گردد.
</div>

In [ ]:
class PoemDataset(torch.utils.data.Dataset):
    def __init__(
            self,
            config,
            device=torch.device('cpu'),
            poet='ferdousi',
            corpus_dir='./Persian_poems_corpus/normalized',
            vocab_path='./vocabulary.txt'
    ):
        self.config = config
        self.device = device
        self.corpus_dir = corpus_dir
        self.vocab_path = vocab_path

        self.words_by_poet = self.load_words(corpus_dir)
        self.vocabulary = self.load_vocabulary()
        self.index_to_word = {index: word for index, word in enumerate(self.vocabulary)}
        self.word_to_index = {word: index for index, word in enumerate(self.vocabulary)}
        self.poet = poet

    def preprocess_lines(self, lines, mask_key):
        lines = [line.strip() for line in lines]
        lines = filter(lambda line: len(line) > 0, lines)
        lines = map(lambda line: line.replace('\n', ''), lines)
        lines = map(lambda line: line.replace('\t', ''), lines)
        lines = map(lambda line: line.replace('\r', ''), lines)
        lines = map(
            lambda index_line:
            f'[BOM_{mask_key}] ' + index_line[1] + ' [EOS]' if index_line[0] % 2 == 1
            else f'[BOM_{mask_key}] ' + index_line[1],
            enumerate(lines)
        )
        words = itertools.chain.from_iterable(map(lambda line: line.split(' '), lines))
        words = filter(lambda word: len(word) > 0, words)
        words = list(words)
        return words

    def load_words(self, corpus_dir):
        words_by_poet = {}
        for filename in os.listdir(corpus_dir):
            with open(os.path.join(corpus_dir, filename)) as f:
                poet_name = filename.split('_')[0]
                lines = f.readlines()
                words_by_poet[poet_name] = self.preprocess_lines(lines, poet_name)
        return words_by_poet

    def load_vocabulary(self):
        with open(self.vocab_path) as f:
            vocabulary = f.readlines()
        vocabulary = [word.strip() for word in vocabulary]
        return vocabulary

    @property
    def all_poets(self):
        return self.words_by_poet.keys()

    @property
    def poet(self):
        return self._poet

    @poet.setter
    def poet(self, poet):
        self._poet = poet
        if poet == 'all':
            self.words = list(itertools.chain.from_iterable(self.words_by_poet.values()))
        else:
            self.words = self.words_by_poet[poet]
        self.words_indexes = [self.word_to_index[w] for w in self.words]

    def __len__(self):
        return len(self.words_indexes) - self.config.sequence_length

    def __getitem__(self, index):
        tensors = (
            torch.tensor(self.words_indexes[index:index + self.config.sequence_length]).to(self.device),
            torch.tensor(self.words_indexes[index + 1:index + self.config.sequence_length + 1]).to(self.device),
        )
        return tensors


dataset = PoemDataset(config, device=torch.device('cpu'), poet='all', corpus_dir=corpus_dir, vocab_path=vocab_path)

for i in range(10):
    print(dataset[i])

(tensor([    1,  2444, 38083,  3899,  2667,  1267, 35473, 32103, 29073,     1]), tensor([ 2444, 38083,  3899,  2667,  1267, 35473, 32103, 29073,     1, 36588]))
(tensor([ 2444, 38083,  3899,  2667,  1267, 35473, 32103, 29073,     1, 36588]), tensor([38083,  3899,  2667,  1267, 35473, 32103, 29073,     1, 36588, 22622]))
(tensor([38083,  3899,  2667,  1267, 35473, 32103, 29073,     1, 36588, 22622]), tensor([ 3899,  2667,  1267, 35473, 32103, 29073,     1, 36588, 22622,   317]))
(tensor([ 3899,  2667,  1267, 35473, 32103, 29073,     1, 36588, 22622,   317]), tensor([ 2667,  1267, 35473, 32103, 29073,     1, 36588, 22622,   317, 30370]))
(tensor([ 2667,  1267, 35473, 32103, 29073,     1, 36588, 22622,   317, 30370]), tensor([ 1267, 35473, 32103, 29073,     1, 36588, 22622,   317, 30370,  3614]))
(tensor([ 1267, 35473, 32103, 29073,     1, 36588, 22622,   317, 30370,  3614]), tensor([35473, 32103, 29073,     1, 36588, 22622,   317, 30370,  3614, 32966]))
(tensor([35473, 32103, 29073,     

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    در کلاس Model مدل این بخش ساخته می‌شود که شامل سه بخش embedding، lstm و یک تابع خطی برای تبدیل خروجی lstm به vocab است. لایه‌های هر بخش توسط pytorch ساخته می‌شوند و مقادیر ابتدایی برابر با صفر است.
</div>

In [ ]:
from torch import nn, optim

class Model(nn.Module):
    def __init__(self, config, device=torch.device('cpu')):
        super(Model, self).__init__()
        self.lstm_size = config.embedding_size
        self.lstm_hidden_size = config.lstm_hidden_size
        self.lstm_dropout = 0.2
        self.embedding_dim = config.embedding_size
        self.num_layers = config.lstm_num_layers
        self.device = device
        self.vocab_size = config.vocab_size
        self.embedding = nn.Embedding(
            num_embeddings=self.vocab_size,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.lstm_size,
            hidden_size=self.lstm_hidden_size,
            num_layers=self.num_layers,
            dropout=self.lstm_dropout,
        )
        self.fc = nn.Linear(self.lstm_size, self.vocab_size)
        self.to(device)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.fc(output)
        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size).to(self.device),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size).to(self.device))


<div dir="rtl"  style ='font-family: "B Nazanin";'>
    نهایتاً یک تابع کلاسیک برای train داده‌ها استفاده می‌شود. در این تابع از optimizer جنریک Adam استفاده شده و loss به روش cross entropy محاسبه شده است. 
</div>

In [ ]:
# run this cell to generate new vocabulary

all_words = list(itertools.chain.from_iterable(dataset.words_by_poet.values()))
word_counts = Counter(all_words)
vocab = sorted(list(word_counts))
with open(vocab_path, 'w') as f:
    for word in vocab:
        f.write(word + '\n')

In [ ]:
from torch.utils.data import DataLoader

def train(dataset, model, config, checkpoint_path='../data/checkpoints', max_epochs=10, ):
    if wandb_active:
        wandb.watch(model)
    model.train()

    dataloader = DataLoader(dataset, batch_size=config.batch_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

    print({'batch_count': len(dataloader), 'epoch_count': max_epochs})
    for epoch in range(max_epochs):
        state_h, state_c = model.init_state(config.sequence_length)
        for batch, (x, y) in enumerate(dataloader):
            optimizer.zero_grad()

            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            loss = criterion(y_pred.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss.backward()
            optimizer.step()
            print({'epoch': epoch, 'batch': batch, 'loss': loss.item()})
            if wandb_active and batch % config.log_interval == 0:
                wandb.log({"loss": loss})
        try:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
            }, os.path.join(checkpoint_path, f'{run_name}_checkpoint_{epoch}_{time.time()}.pt'))
        except Exception as e:
            print(e)

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    از تابع predict برای حدس ادامه‌ی بیت استفاده می‌گردد.<br> دیکودر مدل در این بخش قرار دارد، بطوریکه وزن‌های لازم برای انتخاب کلمه‌ی بعد از اعمال softmax روی hidden state آخرین مرحله حاصل می‌گردند. 
    همچنین برای پیدا کردن مصراعی که قافیه داشتن یا نداشتن آن با فرض اولیه ی ما همخونی دارد به این صورت عمل می‌کنیم که به تعداد کمتر از 
    maximum try
    هر بار با ساخت یک مصراع و بررسی قافیه ی آن نسبت به فرض اولیه در صورتی که تناقض وجود داشت دوباره کل مصراع را تولید می‌کنیم. در غیر اینصورت مصراع را به عنوان خروجی می‌دهیم. 
</div>

In [ ]:
def does_ryhme(first_mesra, sec_mesra):
    first_ghafie = [x for x in filter(lambda x: len(x)>0, first_mesra.split(" "))][-1]
    second_ghafie = [x for x in filter(lambda x: len(x)>0, sec_mesra.split(" "))][-1]
    min_len = min(len(first_ghafie), len(second_ghafie))

    for level in range(-1, -min_len-1, -1):
        if not first_ghafie[level:] == second_ghafie[level:]:
            break
        if level < -1:
            break
    return level!=-1

def predict(dataset, model, text, max_predict_length=12, have_ryhme=True):
    max_try = 10
    if have_ryhme:
        mesra_diff = "[BOM_ferdousi]"
    else:
        mesra_diff = "[BOM_hafez]"
    model.eval()

    words = text.split(' ')
    state_h, state_c = model.init_state(len(words))
    mesras = text.split("[BOM]")
    first_mesra = list(sorted(mesras, key= lambda x: len(x), reverse=True))[0]

    stop = False
    tries = 0
    while not stop and tries < max_try:
        tries += 1
        i = 0
        words = text.split(' ')
        while words[-1] != '[EOS]' and i < max_predict_length:
            x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]]).to(device)
            y_pred, (state_h, state_c) = model(x, (state_h, state_c))

            last_word_logits = y_pred[0][-1]
            p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().cpu().numpy()
            word_index = np.random.choice(len(last_word_logits), p=p)
            words.append(dataset.index_to_word[word_index])
            if words[-1] == '[EOS]':
                ryhmes = does_ryhme(first_mesra, " ".join(words[:-1]))
                if have_ryhme == ryhmes:
                    print("ryhmes match!!")
                    stop = True
                    break
                else:
                    print("ryhmes font match :( starting again")
            i += 1
    return words

<div dir="rtl"  style ='font-family: "B Nazanin";'>
برای ترین مدل از ابتدا خطوط زیر باید اجرا شوند. در غیر این صورت از مدل آماده موجود در این <a href="https://drive.google.com/drive/folders/10dOPSBW6oOljcQ4sZSAqEqV7W7IcNd8e?usp=sharing">لینک</a> استفاده کنید. 
</div>

In [ ]:
model = Model(config, device)
dataset = PoemDataset(config, device=device, poet='all', corpus_dir=corpus_dir, vocab_path=vocab_path)

In [5]:
# train on generic dataset
train(dataset, model, config, checkpoint_path=checkpoints_dir, max_epochs=10)
if wandb_active:
    wandb.finish()

In [ ]:
model_ferdousi = copy.deepcopy(model)
dataset.poet = 'ferdousi'
run_name = 'ferdousi_fine_tune'
if wandb_active:
    wandb.init(project=project_name, name=run_name, reinit=True)
train(dataset, model_ferdousi, config, checkpoint_path=checkpoints_dir, max_epochs=3)
if wandb_active:
    wandb.finish()

{'batch_count': 2803, 'epoch_count': 3}


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:762: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  ../aten/src/ATen/native/cudnn/RNN.cpp:926.)
  self.dropout, self.training, self.bidirectional, self.batch_first)


{'epoch': 0, 'batch': 0, 'loss': 6.889185905456543}
{'epoch': 0, 'batch': 1, 'loss': 6.5529656410217285}
{'epoch': 0, 'batch': 2, 'loss': 6.2994842529296875}
{'epoch': 0, 'batch': 3, 'loss': 5.536078929901123}
{'epoch': 0, 'batch': 4, 'loss': 5.52945613861084}
{'epoch': 0, 'batch': 5, 'loss': 5.632105827331543}
{'epoch': 0, 'batch': 6, 'loss': 5.0263776779174805}
{'epoch': 0, 'batch': 7, 'loss': 4.857870578765869}
{'epoch': 0, 'batch': 8, 'loss': 4.451596260070801}
{'epoch': 0, 'batch': 9, 'loss': 4.313296794891357}
{'epoch': 0, 'batch': 10, 'loss': 4.216029167175293}
{'epoch': 0, 'batch': 11, 'loss': 4.048924922943115}
{'epoch': 0, 'batch': 12, 'loss': 4.257582664489746}
{'epoch': 0, 'batch': 13, 'loss': 4.120383262634277}
{'epoch': 0, 'batch': 14, 'loss': 4.034390449523926}
{'epoch': 0, 'batch': 15, 'loss': 4.085867404937744}
{'epoch': 0, 'batch': 16, 'loss': 4.050901889801025}
{'epoch': 0, 'batch': 17, 'loss': 3.917092800140381}
{'epoch': 0, 'batch': 18, 'loss': 4.214128017425537}
{

In [ ]:
model_hafez = copy.deepcopy(model)
dataset.poet = 'hafez'
run_name = 'hafez_fine_tune'
if wandb_active:
    wandb.init(project=project_name, name=run_name, reinit=True)
train(dataset, model_hafez, config, checkpoint_path=checkpoints_dir, max_epochs=10)
if wandb_active:
    wandb.finish()

In [ ]:
model_moulavi = copy.deepcopy(model)
dataset.poet = 'moulavi'
run_name = 'moulavi_fine_tune'
if wandb_active:
    wandb.init(project=project_name, name=run_name, reinit=True)
train(dataset, model_moulavi, config, checkpoint_path=checkpoints_dir, max_epochs=3)
if wandb_active:
    wandb.finish()

<div dir="rtl"  style ='font-family: "B Nazanin";'>
برای بدست آوردن مصرع دوم هر بیت کافی است مصرع اول را به تابع
get_next_mesra_lstm
بدهید. خروجی مورد نظر پرینت خواهد شد.
این تابع بر اساس وجود داشتن یا نداشتن قافیه یک جدا کننده ی مصرع مخصوص انتخاب کرده و به اول و اخر مصراع اضافه می‌کند. سپس آن را به مدل ایجاد شده داده و خروجی را پرینت می‌کند 
</div>

In [ ]:
def get_next_mesra_lstm(first_mesra, have_ryhme):
    model = Model(config, device)
    chechkpoint = torch.load('./NLP Class/checkpoints/all_poem_train_checkpoint_9_1654559078.2024975.pt',
                            map_location=torch.device('cpu'))
    print(chechkpoint['epoch'])
    model.load_state_dict(chechkpoint['model_state_dict'])
    if have_ryhme:
        mesra_diff = "[BOM_ferdousi]"
    else:
        mesra_diff = "[BOM_hafez]"
    first_mesra = f"{mesra_diff} {first_mesra} {mesra_diff}"
    print('\n'.join(predict(dataset, model, text=first_mesra, have_ryhme=have_ryhme)))

In [ ]:
get_next_mesra_lstm("توانا بود هر که دانا بود", False)

ryhmes match!!
[BOM_hafez] توانا بود هر که دانا بود [BOM_hafez] [BOM_moulavi] اسرار که جزو جان نه که همیشه منزل است [EOS]


<div dir="rtl"  style ='font-family: "B Nazanin";'>
    <h3> 3. مدل بر پایه مکانیسم توجه </h3>
    ابتدا به کمک کلاس SampleDataset پیش‌پردازش‌های لازم را انجام می‌دهیم که شامل خواندن متن و افزودن عبارات اول و پایان مصراع‌ها می‌باشد. هم‌چنین این کلاس تابعی برای بررسی قافیه‌دار بودن یک بیت دارد، بدین ترتیب که کلمات آخر دو مصراع آن را بررسی می‌کند و در صورتیکه دو حرف آخر یکسان داشته باشند، هم‌قافیه‌اند. <br>در تابع __getitem__ با دریافت ایندکس هر بیت، نتیجه‌ی encode شده‌ی آن بیت خروجی داده می‌شود که ورودی مناسب مدل است.
</div>

In [5]:
from transformers import pipeline, AutoTokenizer, GPT2LMHeadModel

base_path =  "./data"

class SampleDataset(torch.utils.data.Dataset):
    def __init__(
            self,
            tokenizer,
            base_path,
            train_path,
            max_epochs = 20,
            batch_size = 256,
            sequence_length = 6,
            log_interval = 10
    ): 
        self.start_mesra = '[BOM] '
        self.end_mesra = '[EOM]'
        self.start_beyt = ''
        self.end_beyt = ' [EOS]'
        self.base_path = base_path
        self.train_path = train_path
        self.sequence_length = sequence_length
        self.beyts = self.load_beyts()
        self.tokenizer = tokenizer
    
    def load_prepared_beyts(self):
        with open(os.path.join(self.base_path, self.train_path)) as fp:
            return fp.readlines()

    def load_beyts(self):

        beyt_file = []
        with open(os.path.join(self.base_path, self.train_path)) as fp:
            lines = fp.readlines()
            for i in tqdm(range(0, len(lines) - 1, 2)):
                mesra1 = self.start_mesra + lines[i].strip() + self.end_mesra
                mesra2 = self.start_mesra + lines[i + 1].strip() + self.end_mesra
                b = self.start_beyt + mesra1.strip() + ' ' + mesra2.strip() + self.end_beyt
                beyt_file.append(b)
        return beyt_file

    def __len__(self):
        return len(self.beyts)

    def does_ryhme(self, beyt):
        mesras = beyt.split(self.end_mesra)
        mesras = [x for x in filter(lambda x: len(x)>0, mesras)]
        first_ghafie = [x for x in filter(lambda x: len(x)>0, mesras[0].split(" "))][-1]
        second_ghafie = [x for x in filter(lambda x: len(x)>0, mesras[1].split(" "))][-1]
        min_len = min(len(first_ghafie), len(second_ghafie))

        for level in range(-1, -min_len-1, -1):
            if not first_ghafie[level:] == second_ghafie[level:]:
                break
            if level < -1:
                break
        return level!=-1


    def __getitem__(self, index):
        current_beyt = self.beyts[index]
        mesras = current_beyt.split(self.end_mesra)
        mesras = [x for x in filter(lambda x: len(x)>0, mesras)]
        first_token = self.tokenizer.encode(mesras[0])
        second_token = self.tokenizer.encode(mesras[1])
        # first_token.append(self.does_ryhme(current_beyt))
        # second_token.append(self.does_ryhme(current_beyt))
        tensors = (
            first_token,
            second_token
        )
        return self.tokenizer.encode(current_beyt.replace(self.end_mesra, "")).append()

In [ ]:
train_path = "train.txt"
test_path = "test.txt"
base_path = "./data"
dataset = SampleDataset(AutoTokenizer.from_pretrained('bolbolzaban/gpt2-persian'), base_path, train_path)
val_dataset = SampleDataset(AutoTokenizer.from_pretrained('bolbolzaban/gpt2-persian'), base_path, test_path)

<div dir="rtl"  style ='font-family: "B Nazanin";'>
     برای ساخت مدل بر پایه مکانیسم توجه، از مدل GPT2 بر پایه <a href="https://huggingface.co/bolbolzaban/gpt2-persian">مدل pre_trained بلبل‌زبان</a> استفاده کردیم. توضیحات این مدل و دیتای مورد استفاده‌ی آن در <a href="https://khashei.medium.com/a-not-so-dangerous-ai-in-the-persian-language-39172a641c84">این لینک</a> قابل مشاهده است. این مدل را بر اساس داده‌های مورد استفاده‌ی خودمان fine tune کردیم.<br> ابتدا این مدل گرفته می‌شود و متغیرهای لایه‌های درونی آن freeze می‌گردند، سپس داده‌ی اشعار پیش‌پردازش شده tokenize می‌شود و به دو بخش train و test تقسیم می‌گردد. برای train کردن، مدل بلبل‌زبان را بر اساس این داده fine tune می‌کنیم.<br>خروجی این مدل یک generator است که برای ساخت مصراع دوم از pipeline استفاده می‌کند که ورودی (مصراع اول) را به ساختاری مناسب ورودی دادن به مدل (با پیش‌پردازش و encoding) تبدیل کرده و به کمک مدل train شده، مصراع دوم را ساخته و decode می‌کند و بدین ترتیب بیت را می‌سازد.<br>کیفیت خروجی به تعداد epochها و سایز batchها بستگی خواهد داشت که افزایش آنها باعث بهبود نتیجه و کاهش سرعت training (بسته به سخت‌افزار سیستم) می‌گردد. ما پس از چندین آزمون و خطا با در نظر گرفتن مقدار 10 برای این دو متغیر به نتایج نسبتاً مناسب در زمان مطلوب رسیدیم.<br>کد این بخش روی gpu اجرا شده و در صورت نبودن gpu در سیستم، بخش مربوط به device در آن تغییر می‌یابد. مدل نهایی این بخش در <a href="https://drive.google.com/drive/folders/1UKqNs67gdAJgNwLtrQd4f2MtsvA4VCGc?usp=sharing">اینجا</a> قرار گرفته و قابل دانلود است.<br>
</div>

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bolbolzaban/gpt2-persian')

In [ ]:
from transformers import (
    AutoTokenizer,
    TextDataset,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    AutoModelWithLMHead,
    BertModel, 
    GPT2LMHeadModel,
    pipeline
)
import torch
import os


class MesraModel:
    def __init__(self, train_path, test_path, model_dir="./model"):
        self.base_path = "./data"
        self.model = None
        self.tokenizer = None
        self.generator = None
        self.train_path = train_path
        self.test_path = test_path
        self.model_dir = model_dir
        self.trainer = None
        # self.cleaner = Cleaner()

    def read_data(self, tokenizer, train_path=None, test_path=None):
        train_path = train_path if train_path is not None else self.train_path
        test_path = test_path if test_path is not None else self.test_path
        train_dataset = SampleDataset(AutoTokenizer.from_pretrained('bolbolzaban/gpt2-persian'), base_path, train_path)
        test_dataset = SampleDataset(AutoTokenizer.from_pretrained('bolbolzaban/gpt2-persian'), base_path, test_path)
        # train_dataset = TextDataset(
        #     tokenizer=tokenizer, file_path=os.path.join(self.base_path, train_path), block_size=128)
        
        # test_dataset = TextDataset(
        #     tokenizer=tokenizer, file_path=os.path.join(self.base_path, test_path), block_size=128)
        
        data_collator = DataCollatorForLanguageModeling(
            tokenizer=tokenizer,
            mlm=False,
        )
        return train_dataset, test_dataset, data_collator

    def read_model(self, model_type='bolbolzaban/gpt2-persian'):
        self.model = GPT2LMHeadModel.from_pretrained(model_type)
        self.tokenizer = AutoTokenizer.from_pretrained('bolbolzaban/gpt2-persian')
        # self.tokenizer.add_tokens(['[EOM]', '[BOM]', '[EOS]'], special_tokens=True)
        return self.model, self.tokenizer


    def freeze_lower_layers(self):
        for param in self.model.base_model.parameters():
            param.requires_grad = False

        for param in (
            self.model.base_model.h[23].parameters() or self.model.base_model.h[22].parameters()
        ):
            param.requires_grad = True
            
    def fine_tune_model(self, model, train_texts, val_texts, data_collator):
        training_args = TrainingArguments(
        output_dir=self.model_dir,
        overwrite_output_dir=True,
        num_train_epochs=12,
        # Set the batch size to a maximum value that could fit into GPU memory,
        # for example 12 is the largest batch size that could work on a 6gb GPU when training the last to layers
        per_device_train_batch_size=12,
        per_device_eval_batch_size=12,
        eval_steps=1000,
        save_steps=1000,
        warmup_steps=500)

        trainer = Trainer(
            model=model,
            args=training_args,
            data_collator=data_collator,
            train_dataset=train_texts,
            eval_dataset=val_texts,
        )
        trainer.train()
        self.trainer = trainer 
        return trainer
    
    def load_model(self):
        model, tokenizer = self.read_model(self.model_dir)
        generator = pipeline('text-generation', model, tokenizer=tokenizer, config={'max_length':256}, device=0)
        self.generator = generator
        return generator
        
    def init_generator(self, used_pretrained=False):      
        if used_pretrained:
            print("in init generator")
            model, tokenizer = self.read_model(self.model_dir)
            print("read model successfully")
            model.to(torch.device("cuda"))
            print("convert model to cuda")
        else:
            model, tokenizer = self.read_model()
            
        self.freeze_lower_layers()
        train_texts, val_texts, data_collator = self.read_data(tokenizer)
        trainer = self.fine_tune_model(model, train_texts, val_texts, data_collator)
        model = trainer.model
    
        generator = pipeline('text-generation', model, tokenizer=tokenizer, config={'max_length':256}, device=0)
        self.generator = generator
        return generator

    def save_model(self, dir=None):
        dir = dir if dir is not None else self.model_dir
        self.trainer.save_model(output_dir=dir)

<div dir="rtl"  style ='font-family: "B Nazanin";'>
برای اعمال رعایت یا عدم رعایت قافیه در این بخش، دو مدل مجزا از هم روی دو نمونه شعر با قافیه (شاهنامه و مثنوی مولوی) و بدون قافیه (غزلیات حافظ و عراقی) train می‌شوند و با توجه به ورودی قافیه، نتیجه مدل مربوطه به عنوان خروجی نمایش داده می‌شود.
</div>

In [ ]:
class MesraGenerator:
    def __init__(self):
        self.notryhme_poets = ["hafez", "eraghi"]
        self.ryhme_poets = ["ferdousi", "moulavi"]
        self.ryhme_model, self.notryhme_model = self.init_model()

    def init_model(self):
        mesra_model = MesraModel("train_path", "test_path", model_dir="./notryhme_model")
        not_ryhme_generator = mesra_model.load_model()

        mesra_model = MesraModel("train_path", "test_path", model_dir="./model")
        ryhme_generator = mesra_model.load_model()
        
        return ryhme_generator, not_ryhme_generator

    def train_all_poets(self):
        ryhme_generator = None
        not_ryhme_generator = None
        for index, poet in enumerate(self.notryhme_poets):
            train_path = f"./data/{poet}_train.txt"
            test_path = f"./data/{poet}_test.txt"
            mesra_model = MesraModel(train_path, test_path, model_dir="./notryhme_model")
            not_ryhme_generator = mesra_model.init_generator(used_pretrained=index!=0)
            mesra_model.save_model("./notryhme_model")

        for index, poet in enumerate(self.ryhme_poets):
            train_path = f"./data/{poet}_train.txt"
            test_path = f"./data/{poet}_test.txt"
            mesra_model = MesraModel(train_path, test_path, model_dir="./ryhme_model")
            ryhme_generator = mesra_model.init_generator(used_pretrained=index!=0)
            mesra_model.save_model("./ryhme_model")
        return ryhme_generator, not_ryhme_generator 
    
    def generate_mesra(self, first_mesra: str, has_ghafie: bool=True):
        first_mesra = f"[BOM] {first_mesra} [BOM]"
        if has_ghafie:
            result = self.ryhme_model(first_mesra)
        else:
            result = self.notryhme_model(first_mesra)

        return result


<div dir="rtl"  style ='font-family: "B Nazanin";'>
بنابراین برای صدا زدن تابع کافی است یک 
MesraGenerator 
ساخته و سپس تابع
generate_mesra
با مصرع مورد نظر صدا زده شود.
</div>

In [ ]:
mesra_generator = MesraGenerator()
mesra_generator.generate_mesra("اشت طره مولوی را در دل شب بی بهانه ")

Setting `pad_token_id` to `eos_token_id`:9 for open-end generation.


[{'generated_text': '[BOM] [BOM]کاشت طره مولوی را در دل شب بی بهانه [BOM] [BOM] با دل مجروح ما کرد آشنا با یک بهانه'}]

<div dir="rtl"  style ='font-family: "B Nazanin";'>
از توابع زیر برای پیش‌پردازش و بازنویسی داده‌های train و test استفاده می‌گردد. هم‌چنین تابع save_not_ryhme_beyts می‌تواند فقط ابیاتی که قافیه ندارند را بیابد و ذخیره کند. این ابیات در ساخت مدل بر پایه ابیات بدون قافیه به ما کمک می‌کنند.
</div>

In [ ]:
def does_ryhme(first_mesra, sec_mesra):
    first_ghafie = [x for x in filter(lambda x: len(x)>0, first_mesra.split(" "))][-1]
    second_ghafie = [x for x in filter(lambda x: len(x)>0, sec_mesra.split(" "))][-1]
    min_len = min(len(first_ghafie), len(second_ghafie))

    for level in range(-1, -min_len-1, -1):
        if not first_ghafie[level:] == second_ghafie[level:]:
            break
        if level < -1:
            break
    return level!=-1

    
def write_to_file(lines, path):
    with open(path, "w") as write_f:
        for line in lines:
            if len(line) > 1:
                write_f.write(line)

def save_not_ryhme_beyts(path):
    out_path = "."+path.split(".")[-2]+"_not_ryhme.txt"
    not_ryhme_lines = list()
    with open(path) as f:
        lines = f.readlines()
        lines = list(filter(lambda x: len(x.strip())>1, lines))
        for i in tqdm(range(0, len(lines) - 1, 2)):
            mesra1 = lines[i].strip()
            mesra2 = lines[i + 1].strip()
            if not does_ryhme(mesra1, mesra2):
                not_ryhme_lines.append(mesra1+os.linesep)
                not_ryhme_lines.append(mesra2+os.linesep)
    write_to_file(not_ryhme_lines, out_path)


def split_filt_to_trian_test():
    file_path = "./data/eraghi_norm_not_ryhme.txt"
    train_perc = 0.8
    with open(file_path) as f:
        lines = f.readlines()
        train_index = math.floor(len(lines)*train_perc)
        train_index = train_index - (train_index%2)
        train_lines = lines[:train_index]
        test_lines = lines[train_index:]
    write_to_file(train_lines, "./data/eraghi_train.txt")
    write_to_file(test_lines, "./data/eraghi_test.txt")


In [5]:
split_filt_to_trian_test()

In [ ]:
save_not_ryhme_beyts("./data/eraghi_norm.txt")